In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 

In [9]:
hmeq=pd.read_csv(r"C:\Users\chemm\Desktop\cours\MOSEF\Python\Exercice\Scoring\Projet_scoring\data\hmeq.csv")
hmeq.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN


In [10]:
description = hmeq.describe()
print("Statistiques descriptives :\n", description)

Statistiques descriptives :
                BAD          LOAN        MORTDUE          VALUE          YOJ  \
count  5960.000000   5960.000000    5442.000000    5848.000000  5445.000000   
mean      0.199497  18607.969799   73760.817200  101776.048741     8.922268   
std       0.399656  11207.480417   44457.609458   57385.775334     7.573982   
min       0.000000   1100.000000    2063.000000    8000.000000     0.000000   
25%       0.000000  11100.000000   46276.000000   66075.500000     3.000000   
50%       0.000000  16300.000000   65019.000000   89235.500000     7.000000   
75%       0.000000  23300.000000   91488.000000  119824.250000    13.000000   
max       1.000000  89900.000000  399550.000000  855909.000000    41.000000   

             DEROG       DELINQ        CLAGE         NINQ         CLNO  \
count  5252.000000  5380.000000  5652.000000  5450.000000  5738.000000   
mean      0.254570     0.449442   179.766275     1.186055    21.296096   
std       0.846047     1.127266    85

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

# Imputation des variables numériques avec la médiane
numerical_cols = ['LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']
hmeq[numerical_cols] = hmeq[numerical_cols].fillna(hmeq[numerical_cols].median())

# Imputation des variables catégorielles avec la valeur la plus fréquente (mode)
categorical_cols = ['JOB', 'REASON']
hmeq[categorical_cols] = hmeq[categorical_cols].fillna(hmeq[categorical_cols].mode().iloc[0])

# Vérification que toutes les valeurs manquantes ont été imputées
print(hmeq.isnull().sum())

# Appliquer Isolation Forest sur les données imputées
iso_forest = IsolationForest(contamination=0.05, random_state=42)
hmeq['is_outlier'] = iso_forest.fit_predict(hmeq[numerical_cols])

# Filtrer les observations qui sont des outliers
outliers = hmeq[hmeq['is_outlier'] == -1]


BAD        0
LOAN       0
MORTDUE    0
VALUE      0
REASON     0
JOB        0
YOJ        0
DEROG      0
DELINQ     0
CLAGE      0
NINQ       0
CLNO       0
DEBTINC    0
dtype: int64
    BAD  LOAN  MORTDUE     VALUE   REASON    JOB   YOJ  DEROG  DELINQ  \
6     1  1800  48649.0   57037.0  HomeImp  Other   5.0    3.0     2.0   
16    1  2200  90957.0  102600.0  HomeImp    Mgr   7.0    2.0     6.0   
31    1  2800  50795.0   63100.0  HomeImp   Self  26.0    2.0    15.0   
32    1  2800   4000.0   60850.0  HomeImp  Other  16.0    4.0     0.0   
36    1  2900   7750.0   67996.0  HomeImp  Other  16.0    3.0     0.0   

         CLAGE  NINQ  CLNO    DEBTINC  is_outlier  
6    77.100000   1.0  17.0  34.818262          -1  
16  122.900000   1.0  22.0  34.818262          -1  
31  145.633333   3.0  45.0  34.818262          -1  
32  112.633333   2.0   9.0  34.818262          -1  
36  122.204663   2.0   8.0  36.211348          -1  


In [5]:
outliers

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,is_outlier
6,1,1800,48649.0,57037.0,HomeImp,Other,5.0,3.0,2.0,77.100000,1.0,17.0,34.818262,-1
16,1,2200,90957.0,102600.0,HomeImp,Mgr,7.0,2.0,6.0,122.900000,1.0,22.0,34.818262,-1
31,1,2800,50795.0,63100.0,HomeImp,Self,26.0,2.0,15.0,145.633333,3.0,45.0,34.818262,-1
32,1,2800,4000.0,60850.0,HomeImp,Other,16.0,4.0,0.0,112.633333,2.0,9.0,34.818262,-1
36,1,2900,7750.0,67996.0,HomeImp,Other,16.0,3.0,0.0,122.204663,2.0,8.0,36.211348,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5938,0,79200,21694.0,103143.0,HomeImp,Other,26.0,0.0,1.0,139.938999,0.0,14.0,31.333685,-1
5939,0,79200,22940.0,110114.0,HomeImp,Other,28.0,0.0,1.0,144.616126,0.0,13.0,32.366512,-1
5940,0,79900,18363.0,109675.0,HomeImp,Other,27.0,0.0,1.0,132.766724,0.0,13.0,31.642335,-1
5941,0,80300,20370.0,109127.0,HomeImp,Other,27.0,0.0,1.0,142.174300,0.0,13.0,33.468763,-1


In [15]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

# Étape 1: Imputer les données manquantes comme avant
numerical_cols = ['LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']
categorical_cols = ['JOB', 'REASON']

hmeq[numerical_cols] = hmeq[numerical_cols].fillna(hmeq[numerical_cols].median())
hmeq[categorical_cols] = hmeq[categorical_cols].fillna(hmeq[categorical_cols].mode().iloc[0])

# Étape 2: Appliquer Isolation Forest sur les colonnes numériques
iso_forest = IsolationForest(contamination=0.05, random_state=42)
hmeq['is_outlier'] = iso_forest.fit_predict(hmeq[numerical_cols])

# Étape 3: Créer une fonction pour détecter les outliers dans chaque colonne (utilisant l'IQR)
def detect_column_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    # Renvoie un masque booléen indiquant si une valeur est un outlier dans cette colonne
    return (df[column] < lower_bound) | (df[column] > upper_bound)

# Étape 4: Détecter les valeurs spécifiques considérées comme des outliers dans chaque colonne
outliers_data = pd.DataFrame()

# Appliquer la détection des outliers (basée sur l'IQR) à toutes les colonnes numériques
for col in numerical_cols:
    outliers_data[col] = detect_column_outliers(hmeq, col)

# Étape 5: Filtrer uniquement les lignes qui sont marquées comme outliers par Isolation Forest
outliers_observations = hmeq[hmeq['is_outlier'] == -1]

# Étape 6: Créer un masque pour afficher uniquement les valeurs qui sont des outliers dans chaque colonne
exact_outliers = outliers_observations[outliers_data.loc[outliers_observations.index]]

# Étape 7: Afficher les valeurs exactes qui sont des outliers
exact_outliers


,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,is_outlier
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6.0,NaN,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,15.0,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5938,NaN,79200.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
5939,NaN,79200.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
5940,NaN,79900.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
5941,NaN,80300.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [16]:
non_null_counts = exact_outliers.count()
non_null_counts.sum()

np.int64(557)